## Install package requirements and import dependencies

In [64]:
!pip install -r requirements.txt --quiet

import openmeteo_requests
import numpy as np
from dotenv import load_dotenv
import pandas as pd
import requests_cache
import subprocess
from retry_requests import retry
from io import StringIO
import hopsworks
import great_expectations as ge
from datetime import date
import json
import time
import statistics
import time
import copy
from pathlib import Path


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


## Load environment variables from the .env file

In [65]:
load_dotenv()

True

## Connect to hopsworks

In [66]:
project = hopsworks.login()

2026-01-05 14:48:12,962 INFO: Closing external client and cleaning up certificates.
Connection closed.
2026-01-05 14:48:12,964 INFO: Initializing external client
2026-01-05 14:48:12,966 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-05 14:48:13,971 WARNING: UserWarning: The installed hopsworks client version 4.3.1 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2026-01-05 14:48:15,067 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1271967


In [67]:
fs = project.get_feature_store()

## Get feature groups

In [68]:
# get shutdown predictions feature group
shutdown_predictions_fg = fs.get_feature_group(
    name='shutdown_predictions',
    version=1
)

# get closed resorts feature group
open_resorts_fg = fs.get_feature_group(
    name='current_resorts',
    version=1
)

In [69]:
# UNCOMMENT ONCE THE SHUTDOWN_PREDICTIONS FEATURE GROUP HAS ROWS
#psw_df = shutdown_predictions_fg.read(dataframe_type="pandas")
#psw_df

In [70]:
# REMOVE ONCE THE SHUTDOWN_PREDICTIONS FEATURE GROUP HAS ROWS
psw_test_elem = {
    "ski_resort_id": [7752047, 1227308055],
    "will_shutdown": [True, False],
    "shutdown_year": [2028, 0]
}
psw_df = pd.DataFrame(psw_test_elem)
psw_df

,ski_resort_id,will_shutdown,shutdown_year
0,7752047,True,2028
1,1227308055,False,0


In [71]:
or_df = open_resorts_fg.read(dataframe_type="pandas")
or_df = or_df.rename(columns={"id": "ski_resort_id"})
or_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.96s) 


,ski_resort_id,name,latitude,longitude
0,1226505097,Torgnon,45.814452,7.554285
1,601135063,Font d'Urle Chaud Clapier,44.910152,5.323491
2,1254287966,Ristolas en Queyras,44.771783,6.960893
3,601115623,Alpe Devero,46.307671,8.252052
4,7752047,San Martino di Castrozza - Passo Rolle,46.268927,11.792439
...,...,...,...,...
834,45409595,Antagnod,45.822300,7.682800
835,1227121146,Gitschenen – Isenthal,46.899355,8.501497
836,601131935,Saint Luc - Chandolin,46.236511,7.625363
837,642545662,Seefeld - Gschwandtkopf,47.317186,11.171586


## Prepare array for writing to markdown table

In [72]:
publish_data_df = psw_df.set_index('ski_resort_id').join(or_df.set_index('ski_resort_id'))
publish_data_df = publish_data_df.iloc[:, [2, 0, 3, 4, 1]]
publish_data_df = publish_data_df.rename(columns={"shutdown_year": "shutdown-year"})
publish_data_df

,name,will_shutdown,latitude,longitude,shutdown-year
ski_resort_id,,,,,
7752047,San Martino di Castrozza - Passo Rolle,True,46.268927,11.792439,2028
1227308055,Euthal,False,47.091908,8.823070,0


In [73]:
# reformat value for all resorts which are not predicted to close down
# in the coming years
publish_data_df = publish_data_df.astype({'shutdown-year': object})
for i in range(0, len(publish_data_df.index)):
    if not publish_data_df.iloc[i, 1]:
        publish_data_df.iloc[i, 4] = ">15 years"

# drop columns which shouldn't be published to the github page
publish_data_df = publish_data_df.drop(columns=['will_shutdown'])
publish_data_df

,name,latitude,longitude,shutdown-year
ski_resort_id,,,,
7752047,San Martino di Castrozza - Passo Rolle,46.268927,11.792439,2028
1227308055,Euthal,47.091908,8.823070,>15 years


In [74]:
def markdown_table(data_df: pd.DataFrame):
    header = list(data_df.columns.values)
    rows = data_df.to_numpy()
    header_row = "| " + " | ".join(map(str, header)) + " |"
    separator_row = "| " + " | ".join("---" for _ in header) + " |"

    body_rows = [
        "| " + " | ".join(map(str, row)) + " |"
        for row in rows
    ]

    return "\n".join([header_row, separator_row, *body_rows])

## Update dashboard file

Creates new table contents and writes them to the markdown dashboard file

In [75]:
root_dir = Path().absolute()
page_file_path = f"{root_dir}/docs/index.md"

table_md = markdown_table(publish_data_df)
file_contents = f"""# Predicted alpine ski resort close-down in the coming years

Will your favorite alpine ski resort close in the next 15 years due to climate change?
Take a look at our predictions in the table below

{table_md}
"""


with open(page_file_path, "w", encoding="utf-8") as f:
    f.write(file_contents)
